In [1]:
from langchain.agents import AgentType, Tool, initialize_agent

# Set up the Search Tool

To use Google Search API, follow the instructions here: https://python.langchain.com/docs/integrations/tools/google_search

Alternatively, use DuckDuckGo Search, https://python.langchain.com/docs/integrations/tools/ddg. It is free and doesn't require any additional set up. However, you may frequently encounter RateLimit errors.

In [35]:
import os

os.environ["GOOGLE_API_KEY"] =
os.environ["GOOGLE_CSE_ID"] =

In [15]:
import os
from langchain.utilities import DuckDuckGoSearchAPIWrapper, GoogleSearchAPIWrapper

google_api_key = os.getenv("GOOGLE_API_KEY")
google_cse_id = os.getenv("GOOGLE_CSE_ID")

search = GoogleSearchAPIWrapper(google_api_key=google_api_key, google_cse_id=google_cse_id)

# search = DuckDuckGoSearchAPIWrapper()

tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world",
    ),
]

In [20]:
# It returns a list of relevant web resources that might or might not directly answer the question.

tools[0].invoke("The current US House Speaker?")

'Congressman Mike Johnson (LA-04) is elected Speaker of the House. Interior of the U.S. Capitol dome. History of the U.S. House. Discover how Congress\xa0... The 56th and current Speaker of the House is Mike Johnson, a Republican from Louisiana. The Speaker of the House is second in line to succeed the President, after the Vice President. Republican Leadership. scalise Majority Leader Results 1 - 10 of 19 ... Vote Question: On Motion to Suspend the Rules and Concur in the Senate Amendment. Bill Title & Description: To amend the Permanent\xa0... Jan 9, 2023 ... The Speaker shall take the Chair on every legislative day precisely at the hour to which the House last ad- journed and immediately call the. The Speaker announced that the House do now adjourn pursuant to section 3(z) of <a target="_blank" rel="bill" href="https://www.congress.gov/bill/118th-congress\xa0... In addition to the women representatives, 7 (4D, 3R) women have served as non-voting delegates representing American Samoa,

# Set up the LLM

Here we use the Anthropic Claude hosted on AWS Bedrock. To access the models, follow https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html

In [4]:
from langchain.llms import Bedrock

llm = Bedrock(
    model_id="anthropic.claude-v2",
)

In [19]:
# Claude's knowledge base is outdated. Nancy Pelosi is no longer the House Speaker.
llm._call("Who is the current US House Speaker?")

" Nancy Pelosi is the current Speaker of the United States House of Representatives. She has held this position since January 2019.\n\nPelosi previously served as Speaker from 2007 to 2011. She is the first woman to hold the position and is currently in her second stint as Speaker. Pelosi represents California's 12th congressional district and has served in the House since 1987. She is a member of the Democratic Party."

# LangChain Agent

An LangChain Agent combines the power of the reasoning capability of LLMs and the information from external tools

In [12]:
agent_chain = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        agent_kwargs = {
            "input_variables": ["input", "agent_scratchpad",]
        }
    )

In [21]:
# The prompt guides the LLM how to think step by step to answer the input query.

print(agent_chain.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Current Search: useful for when you need to answer questions about current events or the current state of the world

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Current Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [27]:
import re

try:
    agent_chain.run(input="Who is the US House Speaker?")
except ValueError as e:
    # Claude doesn't always follow the output format defined in the prompt, which confuses LangChain's output parser.
    # Even though the answers it gives are quite reasonable.
    # This is actually one pain working with LLMs.
    matches = re.match(".+Could not parse LLM output: (`.+`)", str(e))
    print(matches.group(1))



> Entering new AgentExecutor chain...
 Here is my response:

Question: Who is the US House Speaker?

Thought: To find out who the current US House Speaker is, I should perform a web search for this information.

Action: Current Search
Action Input: who is the current us house speaker
Observation: Congressman Mike Johnson (LA-04) is elected Speaker of the House. Interior of the U.S. Capitol dome. History of the U.S. House. Discover how Congress ... The 56th and current Speaker of the House is Mike Johnson, a Republican from ... "Continuity of Government: Current Federal Arrangements and the Future" (PDF). The Speaker of the House is second in line to succeed the President, after the Vice President. Republican Leadership. scalise Majority Leader Results 1 - 10 of 19 ... 1(current); 2. Jan 18, 2024, 05:18 PM | 118th Congress, 2nd Session. Roll Call Number: 19 | Bill Number: H. R. 6914. Vote Question: On ... House Calendar · Days in Session Calendars Show the days on which the House was 

In [33]:
try:
    agent_chain.run(input="How is the weather in San Diego?")
except ValueError as e:
    # print(e)
    matches = re.match(".+Could not parse LLM output: (`.+`)", str(e))
    print(matches.group(1))



> Entering new AgentExecutor chain...
 Here is my attempt to answer the question "How is the weather in San Diego?":

Thought: To determine the current weather in San Diego, I should perform a web search.

Action: Current Search
Action Input: san diego weather
Observation: Be prepared with the most accurate 10-day forecast for San Diego, CA with highs, lows, chance of precipitation from The Weather Channel and Weather.com. San Diego, CA Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days. San Diego, CA Forecast · Morning. 73°. Chance of Rain0% · Afternoon. 70°. Chance of Rain0% · Evening. 51°. Chance of Rain1% · Overnight. 48°. Chance of Rain1% ... Get the latest San Diego weather and 7-day forecast from the FOX 5 San Diego weather team. See the latest doppler radar and weather alerts. 1:00 pm · Feels Like70° · WindWNW 7 mph · Humidity39% · UV Index4 of 11 · Cloud Cover5% · Rain Amount0 in ... San Diego-Downtown CA ... Overnight: Partl

In [26]:
agent_chain.run(input="When and where is the latest meeting between Xi Jingping and Joe Biden?")



> Entering new AgentExecutor chain...
 Here is a response in the requested format:

Question: When and where is the latest meeting between Xi Jingping and Joe Biden?

Thought: To find out when and where the latest meeting between Xi Jinping and Joe Biden took place, I should perform a current events search.

Action: Current Search
Action Input: latest meeting between Xi Jinping and Joe Biden
Observation: Nov 15, 2023 ... Biden, Jr. today held a Summit with President Xi Jinping of the People's Republic of China (PRC), in Woodside, California. The two leaders held ... Nov 16, 2023 ... On November 15 local time, President Xi Jinping and U.S. President Joe Biden held a summit meeting at the Filoli Estate, San Francisco. Nov 16, 2023 ... President Joseph R. Biden, Jr. today held a Summit with President Xi Jinping of the People's Republic of China (PRC), in Woodside, ... Nov 14, 2022 ... On the afternoon of 14 November local time, President Xi Jinping had a meeting with U.S. President Joe 

'The latest meeting between Xi Jinping and Joe Biden was on November 15, 2023 in Woodside, California at the Filoli Estate near San Francisco.'

In [34]:
try:
    agent_chain.run(input="Are there any pandas in San Diego Zoo right now?")
except ValueError as e:
    # print(e)
    matches = re.match(".+Could not parse LLM output: (`.+`)", str(e))
    print(matches.group(1))



> Entering new AgentExecutor chain...
 Here is my response:

Question: Are there any pandas in San Diego Zoo right now?

Thought: To find out if there are currently any pandas at the San Diego Zoo, I should do a web search for recent information on the pandas there.

Action: Current Search
Action Input: san diego zoo pandas

Observation: Nov 16, 2023 ... Panda Bai Yun, 27, walks through her enclosure at the San Diego Zoo on April 16, 2019, days before heading back to China ... In honoring the terms of our conservation loan agreement with the People's Republic of China, 27-year-old Bai Yun and her son, 6-year-old Xiao Liwu, have been ... After years of red tape and tons of application paperwork, the Zoo and China agreed on a 12-year research loan of two giant pandas, Bai Yun and Shi Shi, who ... Nov 16, 2023 ... Chinese President Xi Jinping hinted at a possible return of pandas to the San Diego Zoo and others across country during a visit to ... Nov 9, 2023 ... Bai Yun's mate, Gao Gao